In [4]:
import pandas as pd
import spacy as sp

tokens = pd.read_csv("../data/tokenized_data.csv")
nlp = sp.load("en_core_web_sm")

print("read")

read


In [17]:
def atomic_sentence_extract(doc):
    atomic_sentences = []

    for sentence in doc.sents:
        #extract the subjects and objects and the main verb ROOT
        ##first get the subjects.

        subjects = []
        for tok in sentence.root.lefts:
            if tok.dep_ in ("nsubj", "nsubpass"):
                subjects.append(tok.text)
        if not subjects:
            subjects = ["X"]

        #get the objects/complements
        objects = []
        for tok in sentence.root.rights:
            if tok.dep_ in ("dobj", "attr", "prep", "xcomp"):
                objects.append(tok.text)
        if not objects:
            objects = ["X"]

        #also want to separate relative clauses
        for tok in sentence:
            if tok.dep_ in ("relcl", "advcl", "ccomp"):
                relative_clause = " ".join([t.text for t in tok.subtree])
                atomic_sentences.append(relative_clause)

        #now we want to combine subjs/objs to create the other atomic sentences
        atomic = f"{subjects[0]} {sentence.root.lemma_} {' '.join(objects)}".strip()
        atomic_sentences.append(atomic)

    return atomic_sentences

In [18]:
all_sentences = []

for col, row in tokens.iterrows():
    sentence_id = row["sentence_id"]
    sentence_text = row["sentence_text"]

    doc = nlp(sentence_text)

    atomic_sentences = atomic_sentence_extract(doc)

    for sentence in atomic_sentences:
        all_sentences.append({
            "sentence_id": sentence_id,
            "atomic_sentence": sentence
        })

In [19]:
df_atomic = pd.DataFrame(all_sentences)

df_atomic.to_csv('../data/split_sentences_v1.csv', index=False)

print("Atomic sentences extraction complete. Saved to 'split_sentences_v1.csv'.")

Atomic sentences extraction complete. Saved to 'split_sentences_v1.csv'.
